In [ ]:
from segmentandoDatasets import segmentando_datasets
segmentando_datasets(10000,10000,10000)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.datasets import mnist
import numpy as np
from tensorflow.keras.optimizers import Adam
import keras 


In [ ]:
from preprocessamento import *

csv_file = 'Datasets_csv/df_PUC.csv'
train, teste, val, _, _, _ = preprocessamento(csv_file, 0.6, 0.2, 0.2, True)

In [ ]:
# Tamanho da entrada
input_img = Input(shape=(256, 256, 3))

# Camadas do encoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Camadas do decoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Definir o modelo
autoencoder = Model(input_img, decoded)

In [5]:

checkpoint_path = 'weights_parciais/weights-improvement-{epoch:02d}-{loss:.2f}.weights.h5'
cp_callback = ModelCheckpoint(filepath=checkpoint_path, 
                                  save_weights_only=True, 
                                  monitor='loss',
                                  mode='max', 
                                  save_best_only=True, 
                                  verbose=1)

In [ ]:
autoencoder.compile(optimizer='Adam', loss='mean_squared_error')

history = autoencoder.fit(train,
                epochs=10,
                batch_size=8,
                shuffle=True,
                callbacks=[cp_callback],
                validation_data=(val))

In [ ]:
pd.DataFrame(history.history).plot()

In [8]:
autoencoder.save("Modelos_keras/Autoencoder.keras")
autoencoder.save_weights("weights_finais/Autoencoder.weights.h5")

In [9]:
autoencoder.load_weights("weights_finais/Autoencoder.weights.h5")

In [10]:
encoder = Model(input_img, encoded)
decoder = Model(encoded, decoded)

In [11]:
import matplotlib.pyplot as plt
import numpy as np

def plot_original_and_reconstructed(model, X_val, num_images=5):
    """
    Plota as imagens originais e suas versões reconstruídas pelo autoencoder.
    
    Parâmetros:
        model (keras.Model): O modelo do autoencoder treinado.
        X_val (numpy.ndarray): Conjunto de dados de validação.
        num_images (int): Número de imagens a serem plotadas.
    """
    plt.figure(figsize=(12, 6))

    for i in range(num_images):
        # Seleciona uma imagem aleatória do conjunto de validação
        image_index = np.random.randint(0, len(X_val))
        original_image = X_val[image_index]

        # Passa a imagem original pelo autoencoder para obter a imagem reconstruída
        reconstructed_image = model.predict(original_image.reshape(1, 256, 256, 3))[0]

        # Plota a imagem original e a reconstruída
        plt.subplot(2, num_images, i + 1)
        plt.imshow(original_image)
        plt.title("Original")
        plt.axis("off")

        plt.subplot(2, num_images, i + 1 + num_images)
        plt.imshow(reconstructed_image)
        plt.title("Reconstruída")
        plt.axis("off")

    plt.tight_layout()
    plt.show()

In [12]:
X_val, _ = next(val)

In [ ]:
plot_original_and_reconstructed(autoencoder, X_val)

In [ ]:
from preprocessamento import preprocessamento_dataframe_unico

UFPR04, _ = preprocessamento_dataframe_unico("Datasets_csv/df_UFPR04.csv", True)
UFPR05, _ = preprocessamento_dataframe_unico("Datasets_csv/df_UFPR05.csv", True)

In [15]:
X_UFPR04,_ = next(UFPR04)
X_UFPR05,_ = next(UFPR05)

In [ ]:
plot_original_and_reconstructed(autoencoder, X_UFPR04)
plot_original_and_reconstructed(autoencoder, X_UFPR05)

In [17]:
encoder = Model(inputs=input_img, outputs=encoded)

# Congelando as camadas do encoder para classificação
encoder.trainable = False